# Choropleth Mapping With Folium and PySAL

____
## Author

Serge Rey [sjsrey@gmail.com](sjsrey@gmail)

## Requirements

Since we will be pulling in tiles for basemaps, we need internet connectivity for what follows to work.
____


In [ ]:
from IPython.display import HTML

import pysal as ps
import numpy as np
import pandas as pd

#import matplotlib.pyplot as plt
%matplotlib inline

import folium


## Help functions
In order to have folium maps appear in-line we will need two utility functions.


In [ ]:
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

The source for these functions is an iPython notebook by [Blake Burkhart](http://nbviewer.ipython.org/gist/bburky/7763555/folium-ipython.ipynb).

## OpenStreet Map Tile with Folium

In [ ]:
map = folium.Map(location=[45.52, -122.68], zoom_start=11)
map.simple_marker([45.52, -122.68], popup='Welcome to PySAL @<b>NARSC 2015</b>!')
inline_map(map)

## Base layer for south

In [ ]:
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=4)
map_osm.geo_json(geo_path = 'data/south.json')
embed_map(map_osm)

### Change the zoom level

In [ ]:
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5)
map_osm.geo_json(geo_path = 'data/south.json')
embed_map(map_osm)

## Binding Attribute Data to the Map

In [ ]:
import json

f = open(r'data/south.json')
q = json.load(f)
f.close()

In [ ]:
q.keys()

In [ ]:
features = q['features']
len(features)

In [ ]:
feature_0 = features[0]

In [ ]:
feature_0

In [ ]:
import pandas as pd
indices = []
values = []
for feature in features:
    indices.append(str(feature['properties']['FIPS']))
    values.append(feature['properties']['HR80'])

In [ ]:
df = pd.DataFrame({'HR80': values,
                   'FIPS': indices} )

In [ ]:
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 legend_name='Homicide Rate HR80')
embed_map(map_osm)

## Using PySAL Map Classification Schemes

In [ ]:
y = np.array(df.HR80.tolist())

In [ ]:
import pysal as ps

In [ ]:
ps.version

### Maximum Breaks

In [ ]:
max_breaks = ps.Maximum_Breaks(y, 5)

In [ ]:
bins = max_breaks.bins.tolist()

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=bins[:-1],
                 legend_name='Homicide Rate HR80 (Maximum Breaks)')
embed_map(map_osm)

### Quantiles

In [ ]:
q5 = ps.Quantiles(y, 5).bins.tolist()

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=q5[:-1],
                 legend_name='Homicide Rate HR80 (Quintiles)')
embed_map(map_osm)

In [ ]:
q6 = ps.Quantiles(y, 6).bins.tolist()

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=q6[:-1],
                 legend_name='Homicide Rate HR80 (Quantiles k=6)')
embed_map(map_osm)

### Fisher-Jenks

In [ ]:
fj= ps.Fisher_Jenks(y, 5).bins.tolist()

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=fj[:-1],
                 legend_name='Homicide Rate HR80 (Fisher Jenks)')
embed_map(map_osm)

In [ ]:
ps.esda.mapclassify.Fisher_Jenks_Sampled?